# Libraries

In [1099]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time
from datetime import date
from copy import deepcopy

# Classes


In [1100]:
class Individual:
  """
  A class that represents an individual solution.

  Attributes:
    chromosome (1D list): The chromosome of the individual in [0,1] interval.
    weights (2D list): The weights of the individual in [-1,1] interval.
    bias (1D list): The bias of the individual in [-1,1] interval.
    fitness (float): The fitness of the individual.
  """

  #TODO: Add bias as an attribute of the Individual and tune it as well as the weights and chromosome
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray,  fitness:float = math.inf):
    self._chromosome = chromosome
    self._weights = weights
    self._bias = bias
    
    if fitness < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = fitness
  
  def __str__(self):
    return f"""
    Individual:
        Feature-selection: {self._chromosome}
        Weights: {self._weights}
        Bias: {self._bias}
        Fitness: {self._fitness}
    """
  @property
  def chromosome(self):
    return self._chromosome
  
  @property
  def weights(self):
    return self._weights
  
  @property
  def bias(self):
    return self._bias

  @property
  def fitness(self):
    return self._fitness

  @chromosome.setter
  def chromosome(self, chromosome):
    self._chromosome = chromosome

  @weights.setter
  def weights(self, weights):
    self._weights = weights

  @bias.setter
  def bias(self, bias):
    self._bias = bias

  @fitness.setter
  def fitness(self,f):
    self._fitness = f

  @staticmethod
  def create_individual(K:int = 0, D:int = 0):
    """
    Generates a random individual.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Individual: A random individual.
    """
    chromosome= np.random.randint(low= 0, high= 1 + 1, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Individual(chromosome, weights, bias)

In [1101]:
class Particle(Individual):
  """
  A class that represents an individual solution in Swarm-based problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    best_fitness (float): The best fitness of the individual.
    best_weights_position (numpy.ndarray): The best weight position of the individual.
    best_chromosome_position (numpy.ndarray): The best chromosome position of the individual.
    weights_velocity (numpy.ndarray): The velocity of the weights of the individual.
    chromosome_velocity (numpy.ndarray): The velocity of the chromosome of the individual.  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray, fitness:float = math.inf):
    super().__init__(chromosome, weights, bias, fitness)
    
    self._best_fitness = fitness # At the beginning, the best fitness is the input fitness
    self._best_weights_position = weights # By default, the best weight position is the starting weight position.
    self._best_bias_position = bias # By default, the best bias position is the starting bias position.
    self._best_chromosome_position = chromosome # By default, the best chromosome position is the starting chromosome position.
    self._weights_velocity = np.zeros(weights.shape) # All swarm's weights velocities are set to 0 at the start
    self._bias_velocity = np.zeros(bias.shape) # All swarm's bias velocities are set to 0 at the start
    self._chromosome_velocity = np.zeros(chromosome.shape) # All swarm's chromosome velocities are set to 0 at the start

  def __str__(self):
    return f"""
    Particle:
        Fitness: {self.fitness}
        Actual weights velocity: {self._weights_velocity}
        Actual bias velocity: {self._bias_velocity}
        Actual feature-selection velocity: {self._chromosome_velocity}
        Best fitness: {self._best_fitness}
    """

  @property
  def best_fitness(self):
    return self._best_fitness

  @property
  def best_weights_position(self):
    return self._best_weights_position
  
  @property
  def best_bias_position(self):
    return self._best_bias_position

  @property
  def best_chromosome_position(self):
    return self._best_chromosome_position
  
  @property
  def weights_velocity(self):
    return self._weights_velocity

  @property
  def bias_velocity(self):
    return self._bias_velocity

  @property
  def chromosome_velocity(self):
    return self._chromosome_velocity

  @best_fitness.setter
  def best_fitness(self,f):
    self._best_fitness = f

  @best_weights_position.setter
  def best_weights_position(self,w):
    self._best_weights_position = w

  @best_bias_position.setter
  def best_bias_position(self,b):
    self._best_bias_position = b

  @best_chromosome_position.setter
  def best_chromosome_position(self,c):
    self._best_chromosome_position = c

  @weights_velocity.setter
  def weights_velocity(self,w):
    self._weights_velocity = w

  @bias_velocity.setter
  def bias_velocity(self,b):
    self._bias_velocity = b
    
  @chromosome_velocity.setter
  def chromosome_velocity(self,c):
    self._chromosome_velocity = c

  @staticmethod
  def create_particle(K:int = 0, D:int = 0):
    """
    Generates a random particle.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Particle: A random particle.
    """
    chromosome= np.random.randint(low= 0, high= 1 + 1, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Particle(chromosome, weights, bias)

In [1102]:
class Larvae(Individual):
  """
  A class that represents an individual solution in CRO problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    attempts  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray, fitness:float = math.inf, attempts:int = 0):
    super().__init__(chromosome, weights, bias, fitness)
    self._attempt = attempts
  
  def __str__(self):
    return f"""
    Larvae:
        Fitness: {self.fitness}
        Attempts: {self._attempt}
    """
  
  @property
  def attempts(self):
    return self._attempt
  
  @attempts.setter
  def attempts(self,a):
    self._attempt = a
  
  @staticmethod
  def create_larvae(K:int = 0, D:int = 0, attempts:int = 0):
    """
    Generates a random larvae.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      Attempts (int): The number of attempts.

    Returns:
      Larvae: A random larvae.
    """
    chromosome= np.random.randint(low= 0, high= 1 + 1, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Larvae(chromosome, weights, bias, attempts = attempts)
    

In [1103]:
class Population:
  """
  A class that represents a population of individual solutions.

  Attributes:
  -----------
    size (int): The size of the population.
    genes_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
  
  Methods:
  -----------
    insert_best_gene(Individual): Inserts the best individual of the population into the gene_list.
    add_gene_to_list_at_index(Individual): Adds an individual to the gene_list.
  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    self._size = size
    self._best_gene = None

    if is_empty == True:
      self._genes_list = np.empty(size, dtype=Individual)
    else:
      if is_swarm == True:
        self._genes_list = np.array([Particle.create_particle(K,D) for _ in range(size)]) 
      else:
        self._genes_list = np.array([Individual.create_individual(K,D) for _ in range(size)]) 
  
  def __str__(self):
    return f"""
    Population:
        Size: {self._size}
        Gene list: {self._genes_list}
        Best gene: {self._best_gene}
    """
  @property
  def size(self):
    return self._size
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def best_gene(self):
    return self._best_gene

  @genes_list.setter
  def genes_list(self, genes_list):
    self._genes_list = genes_list
    
  @best_gene.setter
  def best_gene(self,gen):  
    self._best_gene = gen
  
  @staticmethod
  def create_population(size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    """
    Generates a random population.

    Args:
      size (int): The size of the population.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      is_empty (bool): If True, the population will be empty.

    Returns:
      Population: A random population.
    """
    return Population(size, K, D, is_empty, is_swarm)

  def add_gene_to_list_at_index(cls,gen, index:int):
    np.put(cls._genes_list, index, gen)

  def insert_best_gene(cls,gen):
    cls.best_gene = gen
    cls._genes_list[0] = gen  # Insert the best gene at the beginning of the list

In [1104]:
class Swarm(Population):
  """
  A class that represents a population of solutions in Swarm-based problems.

  Attributes:
  -----------
    (same as Population), and additional attributes:

    global_best_fitness (float): The best fitness of the population.
    global_best_weights (numpy.ndarray): The best weight position of the population.
    global_best_chromosome (numpy.ndarray): The best chromosome position of the population.    
  
  Methods:
  -----------

  """

  #FIXME: Cuidado, puede que en vez de Nones tenga que instanciarlos porque en la primera iteración?¿?
  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    super().__init__(size, K, D, is_empty, is_swarm=True)
    self._global_best_fitness = math.inf
    self._global_best_weights = None
    self._global_best_chromosome = None
    self._global_best_bias = None

  def __str__(self):
    return f"""
    Swarm:
        Size: {self._size}
        Particles list: {self.genes_list}
        Best particle: {self.best_gene}
        Global best fitness: {self._global_best_fitness}
        Global best weights: {self._global_best_weights}
        Global best bias: {self._global_best_bias}
        Global best feature-selection: {self._global_best_chromosome}
    """
  @property
  def global_best_fitness(self):
    return self._global_best_fitness

  @property
  def global_best_weights(self):
    return self._global_best_weights

  @property
  def global_best_chromosome(self):
    return self._global_best_chromosome

  @property
  def global_best_bias(self):
    return self._global_best_bias

  @global_best_fitness.setter
  def global_best_fitness(self,f):
    self._global_best_fitness = f
  
  @global_best_weights.setter
  def global_best_weights(self,w):
    self._global_best_weights = w
  
  @global_best_chromosome.setter
  def global_best_chromosome(self,c):
    self._global_best_chromosome = c

  @global_best_bias.setter
  def global_best_bias(self,b):
    self._global_best_bias = b

  @staticmethod
  def create_swarm(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    """
    Generates a random swarm.

    Args:
      size (int): The size of the swarm.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      is_empty (bool): If True, the swarm will be empty.

    Returns:
      Swarm: A random swarm.
    """
    return Swarm(size, K, D, is_empty)

In [1105]:
class Reef:
  """
  A class that represents a population of solutions in Evolutionary-based problems.

  Attributes:
  ----------- 
    size (int): The size of the population.
    free_occupied_rate (float): The rate of occupied individuals in the reef.
    corals_list (numpy.ndarray): The list of corals in the reef.
    best_coral (Individual): The best individual of the population.
    sorted_indexes (numpy.ndarray): The indexes of the sorted corals_list.
    
  Methods:
  -----------
  """

  # TODO: Add array with indexes of corals sorted by fitness (from best to worst)
  def __init__(self, size:int, rate:float, K:int, D:int, larvaes_attempts:int):
    self._size = size
    self._free_occupied_rate = rate
    self._best_coral = None
    self._corals_list = np.full(shape = [size], fill_value = None) # Empty reef
    self._sorted_indexes = np.array([i for i in range(size)]) # Array with indexes of corals
    self._larvaes_attempts = larvaes_attempts

    occupiedHoles = int(size * rate) # Partially occupy the reef
    self._corals_list[:occupiedHoles] = np.array([Larvae.create_larvae(K,D, larvaes_attempts) for _ in range(occupiedHoles)]) # Fill the reef with individuals

  def __str__(self):
    return f"""
    Reef:
        Size: {self._size}
        Free-occupied rate: {self._free_occupied_rate}
        Corals list: {self._corals_list}
        Best coral: {self._best_coral}
    """
  
  @property
  def size(self):
    return self._size

  @property
  def free_occupied_rate(self):
    return self._free_occupied_rate
  
  @property
  def corals_list(self):
    return self._corals_list

  @property
  def best_coral(self):
    return self._best_coral

  @property
  def sorted_indexes(self):
    return self._sorted_indexes

  @property
  def larvaes_attempts(self):
    return self._larvaes_attempts
  
  @corals_list.setter
  def corals_list(self,corals_list):
    self._corals_list = corals_list

  @best_coral.setter
  def best_coral(self,coral):
    self._best_coral = coral
  
  @staticmethod
  def create_reef(size:int, rate:float, K:int, D:int, larvaes_attempts:int):
    """
    Generates a reef.

    Args:
      size (int): The size of the reef.
      rate (float): The rate of occupied corals in the reef.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      Larvae_attempts (int): The number of attempts of a newborn larvae.

    Returns:
      Reef: A reef.
    """
    return Reef(size, rate, K, D, larvaes_attempts)

  def insert_new_larvae_in_hole(cls, larvae: Larvae, hole_index:int):
    cls.corals_list[hole_index] = larvae

  def remove_coral_from_hole(cls, hole_index:int):
    cls.corals_list[hole_index] = None
  
  def sort_by_fitness(cls):
    # Sort best_indexes by fitness from corals in corals_list, if corals_list[i] is None, then corals_list[i] is the worst
    cls._sorted_indexes = np.argsort([coral.fitness if coral is not None else math.inf for coral in cls._corals_list])


# METHODS


In [1106]:
def sum_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.add(mat1, mat2)

def subtract_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.subtract(mat1, mat2)

def dot_multiply(mat1:np.ndarray, mat2:np.ndarray):
  return np.multiply(mat1, mat2)

def multiply_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.matmul(mat1, mat2)

def dot_divide(mat1:np.ndarray, mat2:np.ndarray):
  return np.divide(mat1, mat2)

In [1107]:
def internal_reproduction(gen: Individual):
  # Mutate chromosome
  mutate_chromosome_point = np.random.randint(0, gen.chromosome.shape[0]) # Choose a random feature to mutate
  gen.chromosome[mutate_chromosome_point] = 1 - gen.chromosome[mutate_chromosome_point] # Activate or deactivate the feature
  
  # Mutate weights
  total_rows = gen.weights.shape[0] 
  total_columns = gen.weights.shape[1] 
  mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate
  
  gen.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

  # Mutate bias
  mutate_bias_point = np.random.randint(0, gen.bias.shape[1]) # Choose a random column to mutate
  gen.bias[0,mutate_bias_point] = np.random.uniform(-1,1) # Mutate the column

In [1108]:
def brooding(larvae: Larvae, attempts:int):
    # Mutate chromosome
    mutate_chromosome_point = np.random.randint(0, larvae.chromosome.shape[0]) # Choose a random feature to mutate
    larvae.chromosome[mutate_chromosome_point] = 1 - larvae.chromosome[mutate_chromosome_point] # Activate or deactivate the feature

    # Mutate weights
    total_rows = larvae.weights.shape[0] 
    total_columns = larvae.weights.shape[1] 
    mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate

    larvae.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

    # Mutate bias
    mutate_bias_point = np.random.randint(0, larvae.bias.shape[1]) # Choose a random column to mutate
    larvae.bias[0,mutate_bias_point] = np.random.uniform(-1,1) # Mutate the column

    larvae.attempts = attempts # Reset attempts
    return larvae

In [1109]:
def external_reproduction(father:Individual, mother:Individual):
  crossover_point = np.random.randint(0, father.chromosome.shape[0])
  crossover_column = random.randint(0, father.weights.shape[1])
  
  # Offsprings are created by combining father and mother chromosomes and weights
  offspring1 = Individual( chromosome = np.concatenate((father.chromosome[:crossover_point], mother.chromosome[crossover_point:])), 
                           weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1),
                           bias = np.concatenate((father.bias[:,:crossover_column], mother.bias[:,crossover_column:]), axis=1) )
  
  offspring2 = Individual( chromosome = np.concatenate((mother.chromosome[:crossover_point], father.chromosome[crossover_point:])), 
                           weights = np.concatenate((mother.weights[:,:crossover_column], father.weights[:,crossover_column:]), axis=1),
                           bias = np.concatenate((mother.bias[:,:crossover_column], father.bias[:,crossover_column:]), axis=1) )
                       
  return offspring1, offspring2

In [1110]:
def broadcast(father:Larvae, mother:Larvae, attempts:int):
    crossover_point_chromosome = np.random.randint(0, father.chromosome.shape[0])
    crossover_column_weights = random.randint(0, father.weights.shape[1])
    crossover_point_bias = random.randint(0, father.bias.shape[1])

    # Offsprings are created by combining father and mother chromosomes and weights
    new_larvae = Larvae( chromosome = np.concatenate((father.chromosome[:crossover_point_chromosome], mother.chromosome[crossover_point_chromosome:])), 
                         weights = np.concatenate((father.weights[:,:crossover_column_weights], mother.weights[:,crossover_column_weights:]), axis=1),
                         bias = np.concatenate((father.bias[:,:crossover_point_bias], mother.bias[:,crossover_point_bias:]), axis=1),
                         attempts= attempts )
                        
    return new_larvae

In [1111]:
def roulette_wheel_selection(population:Population): 
  # Create an empty population for the chosen individuals of the Roulette Wheel
  roulettePopulation = Population.create_population(population.size, is_empty=True)
  roulettePopulation.insert_best_gene(population.best_gene) # Make sure we don't lose our best gene in the roulette 

  # Total population fitness (S)
  S = np.sum([individual.fitness for individual in population.genes_list])

  # Population chromosomes' relative probabilities
  rel_prob = [individual.fitness/S for individual in population.genes_list]

  for idx in range(1, roulettePopulation.size): # -1 because we already inserted the best gene
    r = np.random.uniform() 
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.genes_list): 

      r -= rel_prob[index] 
      if r < 0:
        
        roulettePopulation.add_gene_to_list_at_index(individual, idx)
        break
    # end choose rouletted individual loop
  # end roulette population loop
  return roulettePopulation  

In [1112]:
''' This functions reproduces a population.
    It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
    Returns the input population, updated with the crossover and mutated childs '''

def reproduce_population(population: Population, crossover_probability:float, mutation_probability:float):
  childs = np.empty(shape=0, dtype=Individual) # list to store future offsprings
  gene_list_size = len(population.genes_list)

  for index,individual in enumerate(population.genes_list):
    # Generate a random mother if the crossover probability is met
    if(np.random.random() < crossover_probability):
      random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother

      while(random_mother_index == index): # If the mother is the same as the father, choose another one
        random_mother_index = np.random.randint(0,gene_list_size)
      #end while
      
      # Crossover parents
      mother = population.genes_list[random_mother_index]
      offspring1, offspring2 = external_reproduction(individual, mother)

      # Mutate offsprings if probability is over the mutation probability
      if(np.random.random() < mutation_probability): 
        internal_reproduction(offspring1)

      if(np.random.random() < mutation_probability):
        internal_reproduction(offspring1)

      # Add offsprings to the childs list
      childs = np.append(childs, [offspring1, offspring2])
  
  # Extend the population with the new childs
  population.genes_list = np.concatenate( (population.genes_list, childs) )
  return population

In [1113]:
def update_velocities_and_positions_pso(swarm: Swarm, w:float, c1:float, c2:float):
  # FIXME: Bias and for loops reduced to one line
  # Velocities
  r1 = np.random.uniform()
  r2 = np.random.uniform()

  for index, particle in enumerate(swarm.genes_list): # For each particle in the swarm
    # Chromosome loop
    for k in range(particle.chromosome.shape[0]): # For each feature in the chromosome
      # Update chromosome velocity 
      particle.chromosome_velocity[k] = ( 
                                          (w * particle.chromosome_velocity[k]) +
                                          (c1*r1*(particle.best_chromosome_position[k] - particle.chromosome[k])) +
                                          (c2*r2*(swarm.global_best_chromosome[k] - particle.chromosome[k]))
                                        )

      # Prevent velociy from going out of bounds
      if(particle.chromosome_velocity[k] > 6):
        particle.chromosome_velocity[k] = 6
      elif(particle.chromosome_velocity[k] < -6):
        particle.chromosome_velocity[k] = -6

      velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*particle.chromosome_velocity[k]) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1

      # Update to next position
      if(np.random.uniform() < velocityProbability):
        particle.chromosome[k] = 1
      else:
        particle.chromosome[k] = 0
    # end chromosome loop

    # Weight
    particle.weights_velocity = ( 
                                  (w * particle.weights_velocity) +
                                  (c1*r1*(particle.best_weights_position - particle.weights)) +
                                  (c2*r2*(swarm.global_best_weights - particle.weights))
                                )
    particle.weights += particle.weights_velocity # Update to next position
    #particle.weights = np.clip(particle.weights, -1, 1) # Clip all the weights to prevent them from going out of bounds
    
    # Bias
    particle.bias_velocity = ( 
                                (w * particle.bias_velocity) +
                                (c1*r1*(particle.best_bias_position - particle.bias)) +
                                (c2*r2*(swarm.global_best_bias - particle.bias))
                             )

    particle.bias += particle.bias_velocity # Update to next position
    #particle.bias = np.clip(particle.bias, -1, 1) # Clip all the bias to prevent them from going out of bounds
  # end particle loop
  return swarm

In [1114]:
def split_reef_candidates(candidates:list, fraction:float):
    number_of_candidates = len(candidates)
    split_point = int(number_of_candidates * fraction)

    # Split candidates in two lists
    broadcast_candidates = candidates[:split_point]
    brooding_candidates = candidates[split_point:]
    
    # If broadcast candidates are odd, add one more candidate to the list and remove it from the brooding candidates
    if len(broadcast_candidates) % 2 != 0:
        broadcast_candidates.append(candidates[split_point])
        brooding_candidates.remove(candidates[split_point])

    return [broadcast_candidates, brooding_candidates]

# ELM


In [1115]:
# Computes the fitness for a given Individual using Extreme Learning Machine (ELM) model
def compute_individual_fitness(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # W
    individualWeights = dot_multiply(individual.weights, individual.chromosome[:, np.newaxis])

    # Bias
    Bias = individual.bias

    # Amplify the matrix to the size of Xtraining and Xtestwith repmat
    BiasTrainingMatrix = np.matlib.repmat(Bias, trainingX.shape[0], 1)
    BiasTestMatrix = np.matlib.repmat(Bias, testX.shape[0], 1)

    # H (Sigmoide function) 
    H_training = 1 / (1 + np.exp(-(np.dot(trainingX, individualWeights) + BiasTrainingMatrix)))
    H_test = 1 / (1 + np.exp(-(np.dot(testX, individualWeights) + BiasTestMatrix)))

    # Beta with regularization, complete formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    aux = np.dot(H_training.T, H_training)
    delta = np.identity(aux.shape[0]) * 10e-3
    Beta = np.dot(np.linalg.pinv(aux + delta), np.dot(H_training.T, trainingY))

    # Output
    Y_predicted = np.dot(H_test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [1116]:
# Computes and udpates the fitnesses of a given Population 
def compute_population_fitness(population: Population, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for individual in population.genes_list:
    fitness_i = compute_individual_fitness(individual, D, C, trainingX, trainingY, testX, testY)
    individual.fitness = fitness_i # Update fitness

    # Update best gene in Population
    if population.best_gene is None:
      population.best_gene = individual
    else:
      if individual.fitness < population.best_gene.fitness:
        population.best_gene = individual
  # end for

In [1117]:
# Computes and udpates the fitnesses of a given Swarm
def compute_swarm_fitness(swarm: Swarm, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for particle in swarm.genes_list:  
    fitness_i = compute_individual_fitness(particle, D, C, trainingX, trainingY, testX, testY) 
    particle.fitness = fitness_i

    # Update personal bests and global best
    if particle.fitness < particle.best_fitness:
      particle.best_fitness = particle.fitness
      
      if swarm.best_gene is None:
        swarm.best_gene = particle
      else:
        if particle.fitness < swarm.best_gene.fitness:
          swarm.best_gene = particle

  # Set Swarm's best chromosome and weights
  swarm.global_best_chromosome = swarm.best_gene.chromosome
  swarm.global_best_weights = swarm.best_gene.weights
  swarm.global_best_fitness = swarm.best_gene.fitness
  swarm.global_best_bias = swarm.best_gene.bias

In [1118]:
# Computes and udpates the fitnesses of a given Reef
def compute_reef_fitness(reef: Reef, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for index,coral in enumerate(reef.corals_list):  
    if coral is None: # If the coral is empty, skip it
      continue

    fitness_i = compute_individual_fitness(coral, D, C, trainingX, trainingY, testX, testY) 
    coral.fitness = fitness_i

    # Update best gene in Reef
    if reef.best_coral is None:
      reef.best_coral = coral
    else:
      if coral.fitness < reef.best_coral.fitness:
        reef.best_coral = coral

In [1119]:
# Computes and udpates the fitnesses of a given Pool list
def compute_pool_fitness(pool: list, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    for larvae in pool:  
        fitness_i = compute_individual_fitness(larvae, D, C, trainingX, trainingY, testX, testY) 
        larvae.fitness = fitness_i
    # end for   
    return pool

In [1120]:
def train_model_and_output_results(best_weights:np.ndarray, best_chromosome:np.ndarray, best_bias:np.ndarray, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # W
    individualWeights = dot_multiply(best_weights, best_chromosome[:, np.newaxis])

    # Bias
    Bias = best_bias

    # Amplify the matrix to the size of Xtraining and Xtestwith repmat
    BiasTrainingMatrix = np.matlib.repmat(Bias, trainingX.shape[0], 1)
    BiasTestMatrix = np.matlib.repmat(Bias, testX.shape[0], 1)

    # H (Sigmoide function) 
    H_training = 1 / (1 + np.exp(-(np.dot(trainingX, individualWeights) + BiasTrainingMatrix)))
    H_test = 1 / (1 + np.exp(-(np.dot(testX, individualWeights) + BiasTestMatrix)))

    # Beta with regularization, complete formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    aux = np.dot(H_training.T, H_training)
    delta = np.identity(aux.shape[0]) * 10e-3
    Beta = np.dot(np.linalg.pinv(aux + delta), np.dot(H_training.T, trainingY))

    # Prediction
    predicted_labels = np.dot(H_test, Beta)
    predicted_labels = np.round(predicted_labels)

    #print(f"Predicted labels: {predicted_labels}")
    #print(f"Real labels: {testY}")
    
    correct_prediction = 0
    for index,label in enumerate(testY):
      if predicted_labels[index] == label:
        correct_prediction +=1

    CCR = correct_prediction / testY.shape[0]
    return CCR * 100

# BIO-INSPIRED ALGORITHMS


In [1121]:
''' This functions apply the Evolutionary-based Genetic Algorithm to a given population '''
def ga(population: Population, max_generations:int, crossover_prob:float, mutation_prob:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    # Reproduction
    population = reproduce_population(population, crossover_prob, mutation_prob)

    # Compute Fitness
    compute_population_fitness(population, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)
    
    # Roulette Selection
    population = roulette_wheel_selection(population)

    # Continue iterating
    t += 1

    #print(f"Best Fitness in generation {t}: {population.best_gene.fitness}")
  # end while 
  return population

In [1122]:
''' This method apply the Swarm-based algorithm of PSO to a given swarm'''
def pso(swarm: Swarm, max_generations:int, w_max:float,  w_min:float, c1:float, c2:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    #print(f"Generation {t}")
    # Linearly decrease the inertia weight
    w = w_max - (w_max - w_min) * t / max_generations

    # Evaluate Particles' Fitness and update Pb_i and Gb
    compute_swarm_fitness(swarm, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Update Velocity and Position of each particle
    update_velocities_and_positions_pso(swarm, w, c1, c2)

    # Increase step
    t += 1
    
    #print(f"Global best gene fitness {swarm.best_gene.fitness} - at generation {t}")
  # end while
  return swarm

In [1123]:
def larvae_setting(pool:np.ndarray, reef:Reef, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  # Compute pool fitness
  pool = compute_pool_fitness(pool, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

  # Try to settle the larvaes from the pool
  for larvae in pool:
    random_hole = np.random.randint(0, reef.size) # Random hole in the reef
    
    if reef.corals_list[random_hole] is None: # If the hole is empty, settle the larva
      reef.insert_new_larvae_in_hole(larvae, random_hole)
    elif reef.corals_list[random_hole] is not None and larvae.fitness < reef.corals_list[random_hole].fitness:
      reef.insert_new_larvae_in_hole(larvae, random_hole)
    else: # If the hole is not empty and the larva is not better than the coral, the larva decrease 1 attempt
      larvae.attempts -= 1
  # end for
  return reef


In [1124]:
def asexual_reproduction(reef:Reef, f_asexual:float, asexual_probability:float, larvaes_attemps:int):
  # If the probability of asexual reproduction is met, then budding occurs
  if np.random.uniform(0, 1) < asexual_probability:
    # Sort the coral in ascending order of fitness
    reef.sort_by_fitness()

    # Get one random coral from the asexual fraction of the reef
    asexual_fraction = int(reef.size * f_asexual)
    if asexual_fraction < 1:
      asexual_fraction = 1#raise Exception("The asexual fraction must be greater than 0")
    
    random_coral_index = np.random.randint(0, asexual_fraction)

    random_coral = reef.corals_list[reef.sorted_indexes[random_coral_index]]
    while random_coral is None:
      random_coral_index = np.random.randint(0, asexual_fraction)
      random_coral = reef.corals_list[reef.sorted_indexes[random_coral_index]]
    # end while

    random_coral.attempts = larvaes_attemps # Reset larvae's attempts
    return random_coral

In [1125]:
def predation(reef:Reef, f_depredation:float, depredation_probability:float):
  # Sort the coral in ascending order of fitness
  reef.sort_by_fitness()

  # Depredate from the last corals with the index of the sorted list
  depredation_fraction = int(reef.size * f_depredation)
  for index in range(reef.size - 1, reef.size - depredation_fraction - 1, -1):
    if np.random.uniform(0, 1) < depredation_probability:
      reef.corals_list[reef.sorted_indexes[index]] = None
  # end for

  return reef

In [1126]:
def reproduce_reef(reef:Reef, f_broadcast:float):
  pool = np.empty(shape=0, dtype=Individual) # pool of candidates to select from
  reproduction_candidates = [c for c in reef.corals_list if c is not None]

  # Split reproduction sets
  broadcast_set, brooding_set = split_reef_candidates(reproduction_candidates, f_broadcast)

  # Broadcast    
  if len(broadcast_set) > 0: # If broadcast set is not empty, crossover
    while len(broadcast_set) > 0:
      # Select random father and a mother to crossover
      [father_candidate, mother_candidate] = np.random.choice(broadcast_set, 2, replace=False)
      broadcast_larvae = broadcast(father = father_candidate, 
                             mother =  mother_candidate,
                             attempts= reef.larvaes_attempts)

      # Add offspring to the pool
      pool = np.append(pool,broadcast_larvae) # Two parents reproduce only one coral larva
      
      # Remove fathers of the reproduction list
      broadcast_set.remove(father_candidate)
      broadcast_set.remove(mother_candidate)

  #  Brooding
  if len(brooding_set) > 0: # If broadcast set is not empty, mutate every individual
    for c in brooding_set:
      brooding_larvae = brooding(c, reef.larvaes_attempts)
      pool = np.append(pool, brooding_larvae) # Add the mutated coral to the pool

  return pool

    

In [1127]:
def cro(reef:Reef, max_generations: int, f_broadcast:float, f_asexual:float, f_depredation:float, asexual_probability:float, depredation_probability:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  # Evaluate reef at the beginning
  compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

  pool = np.empty(shape=0, dtype=Individual) # pool of candidates to select from
  t = 0
  while t < max_generations:
    # Asexual reproduction
    asexual_candidate = asexual_reproduction(reef, f_asexual, asexual_probability, reef.larvaes_attempts)
    if asexual_candidate is not None: # If there asexual probability was met and there is a candidate, add to the pool
      pool = np.append(pool, asexual_candidate)

    # Broadcast and brooding
    auxiliar_reef = deepcopy(reef)
    reproduction_candidates = reproduce_reef(auxiliar_reef, f_broadcast)
    pool = np.append(pool, reproduction_candidates)

    # Larvae settings
    reef = larvae_setting(pool, reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Predation
    reef = predation(reef, f_depredation, depredation_probability)

    # Fitness Evaluation
    compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Keep larvaes with attempts > 0
    pool = np.array([l for l in pool if l.attempts > 0])
    
    # Increase step
    t += 1
  #end while

  reef.sort_by_fitness() # Final sort
  reef.best_coral = reef.corals_list[reef.sorted_indexes[0]]
  return reef

# Main

In [1128]:
# HYPERPARAMETERS FOR RUNNING SCRIPTS
# GENERAL
# K - declared already as the number of features

# ELM
OPTIMAL_D = 50
OPTIMAL_C = 1e-1

# BIO-INSPIRED ALGORITHMS
POPULATION_SIZE = 2
MAX_GENERATIONS = 2

# GA
crossover_prob = 0.8
mutation_prob = 0.2

# PSO
w_max = 0.72 # inertia weight coefficient (max)
w_min = 0.1 # inertia weight coefficient (min)
c1 = 1.49 # cognitive coefficient
c2 = 1.49 # social coefficient

# CRO
rho0 = 0.8 # free/occupied rate
eta = 3 # larvae's attempts
f_broadcast = 0.98 # broadcast fraction
f_asexual = 0.05 # asexual fraction
asexual_probability = 0.001 # asexual probability
f_depredation = 0.05 # depredation fraction
depredation_probability = 0.01 # predation probability


In [1129]:
if __name__ == "__main__":
    # Set the path to the data directory
    PORTATIL_DEV = False
    if PORTATIL_DEV:
        CSV_PATH = "C:/Users/david/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
    else:
        CSV_PATH = "C:/Users/david/OneDrive/MrRobot/IITV/4/TFG/data-Vito-PC/"
        
    # List all the csv files in the data directory
    data_directory = os.listdir(CSV_PATH) 

    # Create a dataframe with the parameters we want to save as columns
    results_df = pd.DataFrame(columns=['Dataset',
                                       'Generations', 'Population_size', 
                                       'GA', 'PSO', 'CRO', 
                                       'GA_time', 'PSO_time', 'CRO_time', 
                                       'D', 'C',
                                       'Crossover_prob', 'Mutation_prob',
                                       'w_max', 'w_min', 'c1', 'c2',
                                       'rho0', 'eta', 'f_broadcast', 'f_asexual', 'f_depredation', 'depredation_probability'])

    # Instantiate the list of seeds for which we want to run the algorithms
    seeds = [144] # To use: 42 X, 93 X, 144, 245, 546, 1047, 2048, 4549, 9850, 10051

    for s in seeds:
        # Set the global seed 
        np.random.seed(s)
        random.seed(s)

        # Iterate over all the csv files
        for index,csv in enumerate(data_directory):
            if (csv == "HandWriting.csv" or 
                csv == 'image-segmentation.csv' or 
                csv == 'ionosphere.csv' or 
                csv == "optical-recognition-handwritten-digits.csv" or
                csv == "seismic-bumps.csv"):
                continue
            
            # Set the global seed 
            print(f"Executing ELM with seed {s} for dataset: {csv}, {index+1} out of {len(data_directory)}")
            
            # Read CSV
            df = pd.read_csv(CSV_PATH + csv, sep=" ", header=None)
            end = df.shape[1]

            # - - - DATAFRAME CONSTANTS - - - #
            X = df.iloc[:, 0:end-1].values
            Y = df.iloc[:, end-1].values
            
            J = len(np.unique(Y)) # Number of classes
            N,K = X.shape[0],X.shape[1]  # N = number of samples, K = number of features

            # - - - DATAFRAME PARTITIONS - - - #
            X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # Scaling X (min-max normalization)
            X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # First partition with 20% test data
            X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Second partition with 20% validation data

            # GA
            ga_start_execution_time = time.time()

            # Create initial population
            first_population = Population.create_population(POPULATION_SIZE, K, OPTIMAL_D)

            # Run GA
            last_population = ga(population= first_population, 
                                max_generations= MAX_GENERATIONS,
                                crossover_prob= crossover_prob, 
                                mutation_prob= mutation_prob,
                                OPTIMAL_D= OPTIMAL_D,
                                OPTIMAL_C= OPTIMAL_C,
                                trainX= X_trainVal, 
                                trainY= Y_trainVal,
                                testX= X_testVal,
                                testY= Y_testVal)

            # Train model with best individual
            GA_CCR = train_model_and_output_results(best_weights= last_population.best_gene.weights, 
                                                    best_chromosome = last_population.best_gene.chromosome, 
                                                    best_bias= last_population.best_gene.bias,
                                                    D = OPTIMAL_D,
                                                    C = OPTIMAL_C, 
                                                    trainingX = X_train,
                                                    trainingY = Y_train,
                                                    testX = X_test,
                                                    testY = Y_test)
            # End GA
            ga_execution_time = time.time() - ga_start_execution_time

            # PSO
            pso_start_execution_time = time.time()

            # Create initial population
            first_swarm = Population.create_population(POPULATION_SIZE, K, OPTIMAL_D, is_swarm=True)

            # Run PSO
            last_swarm = pso(swarm= first_swarm,
                            max_generations= MAX_GENERATIONS,
                            w_max= w_max, 
                            w_min= w_min,
                            c1= c1,
                            c2= c2,
                            OPTIMAL_D= OPTIMAL_D,
                            OPTIMAL_C= OPTIMAL_C,
                            trainX= X_trainVal,
                            trainY= Y_trainVal,
                            testX= X_testVal,
                            testY= Y_testVal)

            # Train model with best particle
            PSO_CCR = train_model_and_output_results(best_weights= last_swarm.global_best_weights, 
                                                    best_chromosome= last_swarm.global_best_chromosome, 
                                                    best_bias= last_swarm.global_best_bias,
                                                    D= OPTIMAL_D,
                                                    C= OPTIMAL_C, 
                                                    trainingX= X_train,
                                                    trainingY= Y_train,
                                                    testX= X_test,
                                                    testY= Y_test)
            # End PSO
            pso_execution_time = time.time() - pso_start_execution_time 
            

            # CRO
            cro_start_execution_time = time.time()

            # Create initial population
            first_reef = Reef.create_reef(POPULATION_SIZE, rho0, K, OPTIMAL_D, eta)

            # Run CRO
            last_reef = cro(reef= first_reef,
                            max_generations= MAX_GENERATIONS,
                            f_broadcast= f_broadcast,
                            f_asexual= f_asexual,
                            f_depredation= f_depredation,
                            asexual_probability= asexual_probability,
                            depredation_probability= depredation_probability,
                            OPTIMAL_D= OPTIMAL_D,
                            OPTIMAL_C= OPTIMAL_C,
                            trainX= X_trainVal,
                            trainY= Y_trainVal,
                            testX= X_testVal,
                            testY= Y_testVal)

            # Train model with best individual
            CRO_CCR = train_model_and_output_results(best_weights= last_reef.best_coral.weights, 
                                                    best_chromosome = last_reef.best_coral.chromosome, 
                                                    best_bias = last_reef.best_coral.bias,
                                                    D = OPTIMAL_D,
                                                    C = OPTIMAL_C, 
                                                    trainingX = X_train,
                                                    trainingY = Y_train,
                                                    testX = X_test,
                                                    testY = Y_test)

            # End CRO
            cro_execution_time = time.time() - cro_start_execution_time

            csv_results = pd.DataFrame({'Dataset': csv,
                                        'Generations': MAX_GENERATIONS, 'Population_size': POPULATION_SIZE,
                                        'GA': GA_CCR, 'PSO': PSO_CCR, 'CRO': CRO_CCR,
                                        'GA_time': ga_execution_time, 'PSO_time': pso_execution_time, 'CRO_time': cro_execution_time,
                                        'D': OPTIMAL_D, 'C': OPTIMAL_C,
                                        'Crossover_prob': crossover_prob, 'Mutation_prob': mutation_prob,
                                        'w_max': w_max, 'w_min': w_min, 'c1': c1, 'c2': c2,
                                        'rho0': rho0, 'eta': eta, 'f_broadcast': f_broadcast, 'f_asexual': f_asexual, 'f_depredation': f_depredation, 'depredation_probability': depredation_probability
                                    }, index=[0])

            # Concat results of the experiments to the results dataframe
            results_df = pd.concat([results_df, csv_results], ignore_index=True)
                
        # end csv for

        # Save results to excel file
        output_file = f"LOWERTESTING_seed_{s}.xlsx"

        if os.path.isfile(output_file): # File exists so we append the results            
            with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
                results_df.to_excel(writer, sheet_name=f"seed_{s}", index=False) 
        else: # File doesn't exist so we create it
            with pd.ExcelWriter(output_file) as writer:
                results_df.to_excel(writer, sheet_name=f"seed_{s}", index=False)
                
        print(f"Seed {s} finished")
    # end seeds for
    print("All seeds finished, results saved to results.xlsx")
    
#1025m 44s

Executing ELM with seed 93 for dataset: adult.csv, 1 out of 81
Executing ELM with seed 93 for dataset: balance-scale.csv, 2 out of 81
Executing ELM with seed 93 for dataset: balloons-a.csv, 3 out of 81
Executing ELM with seed 93 for dataset: balloons-b.csv, 4 out of 81
Executing ELM with seed 93 for dataset: balloons-c.csv, 5 out of 81
Executing ELM with seed 93 for dataset: balloons-d.csv, 6 out of 81
Executing ELM with seed 93 for dataset: banknote-authentication.csv, 7 out of 81
Executing ELM with seed 93 for dataset: blood-transfusion-service-center.csv, 8 out of 81
Executing ELM with seed 93 for dataset: breast-cancer-wisconsin-diagnostic.csv, 9 out of 81
Executing ELM with seed 93 for dataset: breast-cancer-wisconsin-prognostic.csv, 10 out of 81
Executing ELM with seed 93 for dataset: breast-cancer-wisconsin.csv, 11 out of 81
Executing ELM with seed 93 for dataset: breast-cancer.csv, 12 out of 81
Executing ELM with seed 93 for dataset: car-evaluation.csv, 13 out of 81
Executing E